In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
import eli5 
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score

c:\anaconda\envs\yoo\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\anaconda\envs\yoo\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [4]:
data_dir = Path('../단국대(사용중)')
feature_dir = Path('../단국대(사용중)')
sub_dir = Path('../단국대(사용중)')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
seed = 42

In [5]:
algo_name = 'lgb'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [6]:
df = pd.read_csv(feature_file, index_col=0)
print(df.shape)
df.head()

(400000, 170)


,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,class,u_r,u_g,g_r,g_z,g_i,d_dered_u,d_dered_g,d_dered_r,d_dered_i,d_dered_z,d_dered_ig,d_dered_zg,d_dered_rz,d_dered_iz,d_obs_det,dered_u_max,dered_u_min,dered_u_diff,dered_u_std,dered_u_sum,dered_g_max,dered_g_min,dered_g_diff,dered_g_std,dered_g_sum,dered_r_max,dered_r_min,dered_r_diff,dered_r_std,dered_r_sum,dered_i_max,dered_i_min,dered_i_diff,dered_i_std,dered_i_sum,dered_z_max,dered_z_min,dered_z_diff,dered_z_std,dered_z_sum,d_dered_u_dered_u_diff,...,d_dered_iz_d_dered_i_diff,d_dered_iz_dered_i_diff,d_dered_iz_d_dered_zg_diff,d_dered_iz_d_dered_z_diff,d_dered_iz_dered_z_diff,d_dered_ig_d_dered_i_diff,d_dered_ig_dered_i_diff,d_dered_ig_d_dered_zg_diff,d_dered_ig_d_dered_z_diff,d_dered_ig_dered_z_diff,d_dered_i_dered_i_diff,d_dered_i_d_dered_zg_diff,d_dered_i_d_dered_z_diff,d_dered_i_dered_z_diff,dered_i_d_dered_zg_diff,dered_i_d_dered_z_diff,dered_i_dered_z_diff,d_dered_zg_d_dered_z_diff,d_dered_zg_dered_z_diff,d_dered_z_dered_z_diff,dered_u_dered_g_max_diff,dered_u_dered_g_min_diff,dered_u_dered_g_sum_diff,dered_u_dered_r_max_diff,dered_u_dered_r_min_diff,dered_u_dered_r_sum_diff,dered_u_dered_i_max_diff,dered_u_dered_i_min_diff,dered_u_dered_i_sum_diff,dered_u_dered_z_max_diff,dered_u_dered_z_min_diff,dered_u_dered_z_sum_diff,dered_g_dered_r_max_diff,dered_g_dered_r_min_diff,dered_g_dered_r_sum_diff,dered_g_dered_i_max_diff,dered_g_dered_i_min_diff,dered_g_dered_i_sum_diff,dered_g_dered_z_max_diff,dered_g_dered_z_min_diff,dered_g_dered_z_sum_diff,dered_r_dered_i_max_diff,dered_r_dered_i_min_diff,dered_r_dered_i_sum_diff,dered_r_dered_z_max_diff,dered_r_dered_z_min_diff,dered_r_dered_z_sum_diff,dered_i_dered_z_max_diff,dered_i_dered_z_min_diff,dered_i_dered_z_sum_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,2.9444,1.1898,0.0,4.2545,2.9272,1.3273,3.3972,2.6643,0.9940,0.9961,0.9971,0.9977,0.9982,0.8704,0.8347,1.1210,1.0428,1.0000,23.1243,0.9940,22.1303,15.6485,24.1183,20.2578,0.9961,19.2617,13.6201,21.2539,18.9551,0.9971,17.9580,10.3325,21.0733,17.6321,0.8704,16.7617,8.3312,20.5430,16.9089,0.8347,16.0743,9.2336,18.7418,-22.1303,...,0.0451,-16.5893,0.2081,0.0446,-15.8662,-0.1273,-16.7617,0.0357,-0.1278,-16.0386,-16.6344,0.1630,-0.0005,-15.9112,16.7974,16.6339,0.7232,-0.1635,-16.0743,-15.9108,2.8665,-0.0021,2.8643,4.1691,-0.0031,3.0450,5.4921,0.1236,3.5753,6.2153,0.1593,5.3764,1.3027,-0.0010,0.1806,2.6257,0.1257,0.7109,3.3488,0.1614,2.5121,1.3230,0.1268,0.5303,2.0462,0.1625,2.3315,0.7232,0.0357,1.8012
1,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,0.6931,1.2533,1.0,1.5997,0.9902,0.6096,0.8930,0.7935,0.9943,0.9959,0.9970,0.9976,0.9974,0.9451,0.9379,1.0211,1.0077,1.0000,14.9664,0.9943,13.9721,9.8798,15.9607,14.0045,0.9959,13.0086,9.1985,15.0004,13.4114,0.9970,12.4144,7.1605,15.4294,13.2363,0.9451,12.2911,6.1264,16.1867,13.1347,0.9379,12.1968,7.0247,15.0700,-13.9721,...,0.0102,-12.2285,0.0698,0.0103,-12.1269,-0.0524,-12.2911,0.0073,-0.0523,-12.1895,-12.2387,0.0597,0.0002,-12.1371,12.2984,12.2389,0.1016,-0.0595,-12.1968,-12.1373,0.9619,-0.0016,0.9603,1.5550,-0.0026,0.5313,1.7302,0.0492,-0.2260,1.8317,0.0564,0.8908,0.5931,-0.0010,-0.4290,0.7683,0.0508,-1.1863,0.8698,0.0580,-0.0695,0.1751,0.0518,-0.7573,0.2767,0.0591,0.3594,0.1016,0.0073,1.1167
2,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,1.0986,1.0225,0.0,1.2501,0.9610,0.2891,0.4754,0.4319,0.9894,0.9912,0.9938,0.9953,0.9965,0.9767,0.9751,1.0094,1.0017,1.0000,16.6076,0.9894,15.6183,11.0438,17.5970,15.6866,0.9912,14.6954,10.3912,16.6778,15.4400,0.9938,14.4462,8.3360,17.4433,15.3217,0.9767,14.3450,7.1652,18.2955,15.2961,0.9751,14.3210,8.2621,17.2677,-15.6183,...,0.0063,-14.3201,0.0266,0.0052,-14.2944,-0.0186,-14.3450,0.0016,-0.0197,-14.3193,-14.3264,0.0202,-0.0011,-14.3007,14.3466,14.3252,0.0257,-0.0214,-14.3210,-14.2996,0.9211,-0.0019,0.9192,1.1676,-0.0045,0.1537,1.2859,0.0126,-0.698

In [7]:
y = df[target_col].values[:320000]
df.drop(target_col, axis=1, inplace=True)
trn = df.iloc[:320000]
tst = df.iloc[320000:]
feature_name = df.columns.tolist()
print(y.shape, trn.shape, tst.shape)

(320000,) (320000, 169) (80000, 169)


In [8]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)
print(X_trn.shape, X_val.shape, y_trn.shape, y_val.shape)

(256000, 169) (64000, 169) (256000,) (64000,)


In [9]:
clf = lgb.LGBMClassifier(objective='multiclass',
                         n_estimators=1000,
                         num_leaves=64,
                         learning_rate=0.1,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=1,
                         colsample_bytree=.8,
                         random_state=seed,
                         n_jobs=-1)
clf.fit(X_trn, y_trn,
        eval_set=[(X_val, y_val)],
        eval_metric='multiclass',
        early_stopping_rounds=10)
p_val = clf.predict(X_val)
p_tst = clf.predict(tst)

[1]	valid_0's multi_logloss: 0.841946
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 0.732913
[3]	valid_0's multi_logloss: 0.65206
[4]	valid_0's multi_logloss: 0.58441
[5]	valid_0's multi_logloss: 0.525291
[6]	valid_0's multi_logloss: 0.479579
[7]	valid_0's multi_logloss: 0.440738
[8]	valid_0's multi_logloss: 0.408563
[9]	valid_0's multi_logloss: 0.379972
[10]	valid_0's multi_logloss: 0.354384
[11]	valid_0's multi_logloss: 0.330944
[12]	valid_0's multi_logloss: 0.314288
[13]	valid_0's multi_logloss: 0.29604
[14]	valid_0's multi_logloss: 0.280338
[15]	valid_0's multi_logloss: 0.268198
[16]	valid_0's multi_logloss: 0.257017
[17]	valid_0's multi_logloss: 0.246292
[18]	valid_0's multi_logloss: 0.236836
[19]	valid_0's multi_logloss: 0.228506
[20]	valid_0's multi_logloss: 0.223241
[21]	valid_0's multi_logloss: 0.216741
[22]	valid_0's multi_logloss: 0.210684
[23]	valid_0's multi_logloss: 0.205477
[24]	valid_0's multi_logloss: 0.201315
[25]	valid_0's 

In [10]:
print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')

93.3281%


In [13]:
perm = PermutationImportance(clf, random_state = 42).fit(X_val, y_val) 
eli5.show_weights(perm, top = 170, feature_names = X_val.columns.tolist())

Weight,Feature
0.4406 ± 0.0038,redshift
0.0390 ± 0.0020,dered_r_dered_i_diff
0.0208 ± 0.0005,dered_u_dered_r_diff
0.0097 ± 0.0007,dered_g_dered_r_sum_diff
0.0088 ± 0.0012,dered_u_dered_g_diff
0.0073 ± 0.0004,dered_u_dered_r_sum_diff
0.0028 ± 0.0003,dered_u_d_dered_zg_diff
0.0024 ± 0.0004,dered_r_dered_i_max_diff
0.0024 ± 0.0002,dered_g_d_dered_z_diff
0.0022 ± 0.0006,d_dered_rz_d_dered_iz_diff
